# 05 - Create list of pests, crop and bird of the selected area

### Description

The goal of this notebook is to:
- determine if birds prey the pests identified for the crops in the area

### Inputs

- list of bird species from the occurrence table, prepared in step 03
- list of pests for the crops in the selected region, prepared in step 04

### External resources 

- API service of Global Biotic Interactions https://github.com/globalbioticinteractions/globalbioticinteractions/wiki/API#interactions

### Outputs

- List of pests' taxonomy, the corresponding hosting crop, and bird species, with indication if they eat that pest

In [1]:
# import modules
import pandas as pd
import requests
%matplotlib inline

## 1. Read list of bird species

We will open the list of species occurrences created in step 03 of the workflow. The table also contains columns with the trait data. We will filter to obtain only bird species that use crops as feeding areas.

In [2]:
# load eBird selected occurrence data
occ_file = '../process_data/eBird_sel.csv'

pdf_o = pd.read_csv(occ_file, low_memory=False)

In [3]:
bird_data_name = pdf_o

# filter species with traits classified, to obtain species that use crops as feeding areas
options = [0, 1]
bird_data_name = bird_data_name[bird_data_name['Annual_crops'].isin(options)]
bird_data_name.head()

,Unnamed: 0.1,Unnamed: 0,GLOBAL UNIQUE IDENTIFIER,LAST EDITED DATE,TAXONOMIC ORDER_x,CATEGORY,TAXON CONCEPT ID_x,COMMON NAME,SCIENTIFIC NAME_x,SUBSPECIES COMMON NAME,...,Annual_crops,Permanent_crops,Proportion_invertebrates_diet,_merge,NativeToValCal,LOCALITY_y,LOCALITY ID_y,geometry_y,stats_cover,area_buff
0,0,0,URN:CornellLabOfOrnithology:EBIRD:OBS1446126022,2022-06-02 18:22:45.475308,26451,species,avibase-603194D3,Bewick's Wren,Thryomanes bewickii,NaN,...,1.0,1.0,1.0,both,1.0,"45491 Reliz Canyon Road, Greenfield, Californi...",L19350044,POLYGON ((-2224592.5060787485 1766274.99430329...,"{28: 7, 61: 2, 69: 8, 111: 1, 121: 59, 122: 8,...",195.3
1,1,1926,URN:CornellLabOfOrnithology:EBIRD:OBS1446126014,2022-06-02 18:22:45.475308,11494,species,avibase-20C2214E,American Kestrel,Falco sparverius,NaN,...,0.0,1.0,0.3,both,1.0,"45491 Reliz Canyon Road, Greenfield, Californi...",L19350044,POLYGON ((-2224592.5060787485 1766274.99430329...,"{28: 7, 61: 2, 69: 8, 111: 1, 121: 59, 122: 8,...",195.3
2,2,2608,URN:CornellLabOfOrnithology:EBIRD:OBS1446126023,2022-06-02 18:22:45.475308,20505,species,avibase-151C2B3F,California Scrub-Jay,Aphelocoma californica,NaN,...,0.0,1.0,0.4,both,1.0,"45491 Reliz Canyon Road, Greenfield, Californi...",L19350044,POLYGON ((-2224592.5060787485 1766274.99430329...,"{28: 7, 61: 2, 69: 8, 111: 1, 121: 59, 122: 8,...",195.3
3,3,9726,URN:CornellLabOfOrnithology:EBIRD:OBS1446126018,2022-06-02 18:22:45.475308,11371,species,avibase-75600969,Northern Flicker,Colaptes auratus,NaN,...,1.0,1.0,1.0,both,1.0,"45491 Reliz Canyon Road, Greenfield, Californi...",L19350044,POLYGON ((-2224592.5060787485 1766274.99430329...,"{28: 7, 61: 2, 69: 8, 111: 1, 121: 59, 122: 8,...",195.3
4,4,10252,URN:CornellLabOfOrnithology:EBIRD:OBS1446126013,2022-06-02 18:22:45.475308,20762,species,avibase-9E9F2476,American Crow,Corvus brachyrhynchos,NaN,...,0.0,1.0,0.4,both,1.0,"45491 Reliz Canyon Road, Greenfield, Californi...",L19350044,POLYGON ((-2224592.5060787485 1766274.99430329...,"{28: 7, 61: 2, 69: 8, 111: 1, 121: 59, 122: 8,...",195.3


In [4]:
# list of species occurrences
#in_file = '../process_data/eBird_sel.csv'
#df_birds = pd.read_csv(in_file, low_memory=False)
df_unique_birds = bird_data_name[['TAXON CONCEPT ID_x', 'SCIENTIFIC NAME_x', 'ORDER1']].drop_duplicates()
df_unique_birds

,TAXON CONCEPT ID_x,SCIENTIFIC NAME_x,ORDER1
0,avibase-603194D3,Thryomanes bewickii,Passeriformes
1,avibase-20C2214E,Falco sparverius,Falconiformes
2,avibase-151C2B3F,Aphelocoma californica,Passeriformes
3,avibase-75600969,Colaptes auratus,Piciformes
4,avibase-9E9F2476,Corvus brachyrhynchos,Passeriformes
...,...,...,...
43061,avibase-70B9DE8D,Turdus migratorius,Passeriformes
45018,avibase-25C6A911,Sayornis nigricans,Passeriformes
53571,avibase-7FB66FEB,Cypseloides niger,Caprimulgiformes
78829,avibase-A7CCACA4,Megascops kennicottii,Strigiformes


## 2. Read list of pests and combine with bird species

We will open the list of pests created at step 04. The pest is considered at order taxnonomic level. Afterwards, combine with species to obtain all possible combinations.

In [5]:
# read list of pests for the crops existing in the area
# set the taxon rank of the pest at the list of columns of df_pests

in_file1 = '../process_data/eppo_crop_pest_data.csv'
eppo_crop_pest_data = pd.read_csv(in_file1, low_memory=False)
df_pests = eppo_crop_pest_data[['eppocode species', 'prefname order', 'crop_eppocode']].drop_duplicates()
df_pests_order = df_pests

In [6]:
df_pests_order

,eppocode species,prefname order,crop_eppocode
0,APLOBE,Rhabditida,ZEAMX
1,APLOBE,Rhabditida,ORYSA
2,APLOBE,Rhabditida,IPOBA
3,APLOBE,Rhabditida,ALLCE
4,APLOBE,Rhabditida,CPSAN
...,...,...,...
2335,RHIOHI,Hemiptera,PUNGR
2336,ANTHBI,Coleoptera,FRAAN
2337,ANTHSI,Coleoptera,FRAAN
2338,ANTHSI,Coleoptera,VACSS


In [7]:
# Combine data between the two dataframes
cross_data = pd.merge(df_unique_birds, df_pests_order, how='cross')
bird_prey = cross_data[['TAXON CONCEPT ID_x', 'SCIENTIFIC NAME_x', 'prefname order', 'eppocode species']].drop_duplicates()

In [8]:
bird_prey

,TAXON CONCEPT ID_x,SCIENTIFIC NAME_x,prefname order,eppocode species
0,avibase-603194D3,Thryomanes bewickii,Rhabditida,APLOBE
6,avibase-603194D3,Thryomanes bewickii,Rhabditida,HETDZE
13,avibase-603194D3,Thryomanes bewickii,Rhabditida,PUNCCH
14,avibase-603194D3,Thryomanes bewickii,Rhabditida,DITYDE
19,avibase-603194D3,Thryomanes bewickii,Rhabditida,DITYDI
...,...,...,...,...
201234,avibase-AB1CB216,Bubulcus ibis,Hemiptera,AONDOR
201235,avibase-AB1CB216,Bubulcus ibis,Hemiptera,RHIOHI
201236,avibase-AB1CB216,Bubulcus ibis,Coleoptera,ANTHBI
201237,avibase-AB1CB216,Bubulcus ibis,Coleoptera,ANTHSI


There is a total of 87648 possible combinations between bird species, pests and crops.

Create a list with unique combinations of birds and possible pest preys:

In [9]:
bird_prey_unique = bird_prey[['SCIENTIFIC NAME_x', 'prefname order']].drop_duplicates()
bird_prey_unique

,SCIENTIFIC NAME_x,prefname order
0,Thryomanes bewickii,Rhabditida
130,Thryomanes bewickii,Dorylaimida
158,Thryomanes bewickii,Architaenioglossa
219,Thryomanes bewickii,Sigmurethra
274,Thryomanes bewickii,Lepidoptera
...,...,...
194515,Cypseloides niger,Coleoptera
194589,Cypseloides niger,Acarida
194790,Cypseloides niger,Thysanoptera
195259,Cypseloides niger,Hymenoptera


In [10]:
print('Number of bird species:',len(bird_prey_unique['SCIENTIFIC NAME_x'].unique()))
print('Number of pest orders:',len(bird_prey_unique['prefname order'].unique()))

Number of bird species: 63
Number of pest orders: 11


There are 1364 combinations between 124 bird species and 11 pest orders.

## 3. Get interaction data from GLOBI
 
- Request data from GLOBI API to determine if birds prey on pests.

In [11]:
'''
# define a function to get data from GLOBI.
# Returns 0 if it is not eaten, 1 otherwise
# inputs
   - predator: bird species
   - prey: pest order
'''
        
base_url = 'https://api.globalbioticinteractions.org/taxon/'
interaction = '/eats/'

def get_globi(pred, prey):
    url = base_url+pred+interaction+prey
    print(url)
    response = requests.get(url)
    data = response.json()
    if len(data['data']) > 0:
        return 1
    else:
        return 0 

In [12]:
%time
# Get data from GLOBI about birds eating preys


# add bolean column to indicate if bird eats pest
bird_prey_unique['eats'] = bird_prey_unique.apply(lambda x: get_globi(x['SCIENTIFIC NAME_x'], \
                                                                      x['prefname order']), axis=1)
bird_prey_unique.head()

CPU times: user 8 µs, sys: 2 µs, total: 10 µs
Wall time: 18.1 µs
https://api.globalbioticinteractions.org/taxon/Thryomanes bewickii/eats/Rhabditida
https://api.globalbioticinteractions.org/taxon/Thryomanes bewickii/eats/Dorylaimida
https://api.globalbioticinteractions.org/taxon/Thryomanes bewickii/eats/Architaenioglossa
https://api.globalbioticinteractions.org/taxon/Thryomanes bewickii/eats/Sigmurethra
https://api.globalbioticinteractions.org/taxon/Thryomanes bewickii/eats/Lepidoptera
https://api.globalbioticinteractions.org/taxon/Thryomanes bewickii/eats/Diptera
https://api.globalbioticinteractions.org/taxon/Thryomanes bewickii/eats/Coleoptera
https://api.globalbioticinteractions.org/taxon/Thryomanes bewickii/eats/Acarida
https://api.globalbioticinteractions.org/taxon/Thryomanes bewickii/eats/Thysanoptera
https://api.globalbioticinteractions.org/taxon/Thryomanes bewickii/eats/Hymenoptera
https://api.globalbioticinteractions.org/taxon/Thryomanes bewickii/eats/Hemiptera
https://api.glob

https://api.globalbioticinteractions.org/taxon/Petrochelidon pyrrhonota/eats/Dorylaimida
https://api.globalbioticinteractions.org/taxon/Petrochelidon pyrrhonota/eats/Architaenioglossa
https://api.globalbioticinteractions.org/taxon/Petrochelidon pyrrhonota/eats/Sigmurethra
https://api.globalbioticinteractions.org/taxon/Petrochelidon pyrrhonota/eats/Lepidoptera
https://api.globalbioticinteractions.org/taxon/Petrochelidon pyrrhonota/eats/Diptera
https://api.globalbioticinteractions.org/taxon/Petrochelidon pyrrhonota/eats/Coleoptera
https://api.globalbioticinteractions.org/taxon/Petrochelidon pyrrhonota/eats/Acarida
https://api.globalbioticinteractions.org/taxon/Petrochelidon pyrrhonota/eats/Thysanoptera
https://api.globalbioticinteractions.org/taxon/Petrochelidon pyrrhonota/eats/Hymenoptera
https://api.globalbioticinteractions.org/taxon/Petrochelidon pyrrhonota/eats/Hemiptera
https://api.globalbioticinteractions.org/taxon/Tachycineta thalassina/eats/Rhabditida
https://api.globalbioticinte

https://api.globalbioticinteractions.org/taxon/Psaltriparus minimus/eats/Architaenioglossa
https://api.globalbioticinteractions.org/taxon/Psaltriparus minimus/eats/Sigmurethra
https://api.globalbioticinteractions.org/taxon/Psaltriparus minimus/eats/Lepidoptera
https://api.globalbioticinteractions.org/taxon/Psaltriparus minimus/eats/Diptera
https://api.globalbioticinteractions.org/taxon/Psaltriparus minimus/eats/Coleoptera
https://api.globalbioticinteractions.org/taxon/Psaltriparus minimus/eats/Acarida
https://api.globalbioticinteractions.org/taxon/Psaltriparus minimus/eats/Thysanoptera
https://api.globalbioticinteractions.org/taxon/Psaltriparus minimus/eats/Hymenoptera
https://api.globalbioticinteractions.org/taxon/Psaltriparus minimus/eats/Hemiptera
https://api.globalbioticinteractions.org/taxon/Molothrus ater/eats/Rhabditida
https://api.globalbioticinteractions.org/taxon/Molothrus ater/eats/Dorylaimida
https://api.globalbioticinteractions.org/taxon/Molothrus ater/eats/Architaenioglos

https://api.globalbioticinteractions.org/taxon/Geococcyx californianus/eats/Sigmurethra
https://api.globalbioticinteractions.org/taxon/Geococcyx californianus/eats/Lepidoptera
https://api.globalbioticinteractions.org/taxon/Geococcyx californianus/eats/Diptera
https://api.globalbioticinteractions.org/taxon/Geococcyx californianus/eats/Coleoptera
https://api.globalbioticinteractions.org/taxon/Geococcyx californianus/eats/Acarida
https://api.globalbioticinteractions.org/taxon/Geococcyx californianus/eats/Thysanoptera
https://api.globalbioticinteractions.org/taxon/Geococcyx californianus/eats/Hymenoptera
https://api.globalbioticinteractions.org/taxon/Geococcyx californianus/eats/Hemiptera
https://api.globalbioticinteractions.org/taxon/Sitta carolinensis/eats/Rhabditida
https://api.globalbioticinteractions.org/taxon/Sitta carolinensis/eats/Dorylaimida
https://api.globalbioticinteractions.org/taxon/Sitta carolinensis/eats/Architaenioglossa
https://api.globalbioticinteractions.org/taxon/Sitta

https://api.globalbioticinteractions.org/taxon/Pica nuttalli/eats/Sigmurethra
https://api.globalbioticinteractions.org/taxon/Pica nuttalli/eats/Lepidoptera
https://api.globalbioticinteractions.org/taxon/Pica nuttalli/eats/Diptera
https://api.globalbioticinteractions.org/taxon/Pica nuttalli/eats/Coleoptera
https://api.globalbioticinteractions.org/taxon/Pica nuttalli/eats/Acarida
https://api.globalbioticinteractions.org/taxon/Pica nuttalli/eats/Thysanoptera
https://api.globalbioticinteractions.org/taxon/Pica nuttalli/eats/Hymenoptera
https://api.globalbioticinteractions.org/taxon/Pica nuttalli/eats/Hemiptera
https://api.globalbioticinteractions.org/taxon/Leiothlypis celata/eats/Rhabditida
https://api.globalbioticinteractions.org/taxon/Leiothlypis celata/eats/Dorylaimida
https://api.globalbioticinteractions.org/taxon/Leiothlypis celata/eats/Architaenioglossa
https://api.globalbioticinteractions.org/taxon/Leiothlypis celata/eats/Sigmurethra
https://api.globalbioticinteractions.org/taxon/Le

https://api.globalbioticinteractions.org/taxon/Agelaius tricolor/eats/Coleoptera
https://api.globalbioticinteractions.org/taxon/Agelaius tricolor/eats/Acarida
https://api.globalbioticinteractions.org/taxon/Agelaius tricolor/eats/Thysanoptera
https://api.globalbioticinteractions.org/taxon/Agelaius tricolor/eats/Hymenoptera
https://api.globalbioticinteractions.org/taxon/Agelaius tricolor/eats/Hemiptera
https://api.globalbioticinteractions.org/taxon/Contopus cooperi/eats/Rhabditida
https://api.globalbioticinteractions.org/taxon/Contopus cooperi/eats/Dorylaimida
https://api.globalbioticinteractions.org/taxon/Contopus cooperi/eats/Architaenioglossa
https://api.globalbioticinteractions.org/taxon/Contopus cooperi/eats/Sigmurethra
https://api.globalbioticinteractions.org/taxon/Contopus cooperi/eats/Lepidoptera
https://api.globalbioticinteractions.org/taxon/Contopus cooperi/eats/Diptera
https://api.globalbioticinteractions.org/taxon/Contopus cooperi/eats/Coleoptera
https://api.globalbioticinter

https://api.globalbioticinteractions.org/taxon/Chordeiles acutipennis/eats/Coleoptera
https://api.globalbioticinteractions.org/taxon/Chordeiles acutipennis/eats/Acarida
https://api.globalbioticinteractions.org/taxon/Chordeiles acutipennis/eats/Thysanoptera
https://api.globalbioticinteractions.org/taxon/Chordeiles acutipennis/eats/Hymenoptera
https://api.globalbioticinteractions.org/taxon/Chordeiles acutipennis/eats/Hemiptera
https://api.globalbioticinteractions.org/taxon/Melanerpes lewis/eats/Rhabditida
https://api.globalbioticinteractions.org/taxon/Melanerpes lewis/eats/Dorylaimida
https://api.globalbioticinteractions.org/taxon/Melanerpes lewis/eats/Architaenioglossa
https://api.globalbioticinteractions.org/taxon/Melanerpes lewis/eats/Sigmurethra
https://api.globalbioticinteractions.org/taxon/Melanerpes lewis/eats/Lepidoptera
https://api.globalbioticinteractions.org/taxon/Melanerpes lewis/eats/Diptera
https://api.globalbioticinteractions.org/taxon/Melanerpes lewis/eats/Coleoptera
http

,SCIENTIFIC NAME_x,prefname order,eats
0,Thryomanes bewickii,Rhabditida,0
130,Thryomanes bewickii,Dorylaimida,0
158,Thryomanes bewickii,Architaenioglossa,0
219,Thryomanes bewickii,Sigmurethra,0
274,Thryomanes bewickii,Lepidoptera,1


In [13]:
# save to file
bird_prey_unique.to_csv('../process_data/bird_prey_unique.csv')

In [14]:
# merge the new classification with the table of the taxonomic classification of the pests
bird_prey_crop = pd.merge(eppo_crop_pest_data, bird_prey_unique, left_on=['prefname order'], right_on=['prefname order'],  how='left')

In [15]:
# save to file
bird_prey_crop.to_csv('../process_data/bird_prey_crop.csv')
bird_prey_crop

,Unnamed: 0,eppocode kingdom,eppocode phylum,eppocode subphylum,eppocode class,eppocode order,eppocode suborder,eppocode family,eppocode genus,eppocode species,...,prefname class,prefname order,prefname suborder,prefname family,prefname genus,prefname species,eppocode,crop_eppocode,SCIENTIFIC NAME_x,eats
0,0,1ANIMK,1NEMAP,NaN,1CHROC,1RHABO,NaN,1APLOF,1APLOG,APLOBE,...,Chromadorea,Rhabditida,NaN,Aphelenchoididae,Aphelenchoides,Aphelenchoides besseyi,APLOBE,ZEAMX,Thryomanes bewickii,0
1,0,1ANIMK,1NEMAP,NaN,1CHROC,1RHABO,NaN,1APLOF,1APLOG,APLOBE,...,Chromadorea,Rhabditida,NaN,Aphelenchoididae,Aphelenchoides,Aphelenchoides besseyi,APLOBE,ZEAMX,Falco sparverius,1
2,0,1ANIMK,1NEMAP,NaN,1CHROC,1RHABO,NaN,1APLOF,1APLOG,APLOBE,...,Chromadorea,Rhabditida,NaN,Aphelenchoididae,Aphelenchoides,Aphelenchoides besseyi,APLOBE,ZEAMX,Aphelocoma californica,0
3,0,1ANIMK,1NEMAP,NaN,1CHROC,1RHABO,NaN,1APLOF,1APLOG,APLOBE,...,Chromadorea,Rhabditida,NaN,Aphelenchoididae,Aphelenchoides,Aphelenchoides besseyi,APLOBE,ZEAMX,Colaptes auratus,0
4,0,1ANIMK,1NEMAP,NaN,1CHROC,1RHABO,NaN,1APLOF,1APLOG,APLOBE,...,Chromadorea,Rhabditida,NaN,Aphelenchoididae,Aphelenchoides,Aphelenchoides besseyi,APLOBE,ZEAMX,Corvus brachyrhynchos,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147415,2339,1ANIMK,1ARTHP,1HEXAQ,1INSEC,1HEMIO,1HETER,1TINGF,1STEPG,STEPTA,...,Insecta,Hemiptera,Heteroptera,Tingidae,Stephanitis,Stephanitis takeyai,STEPTA,VACSS,Pooecetes gramineus,1
147416,2339,1ANIMK,1ARTHP,1HEXAQ,1INSEC,1HEMIO,1HETER,1TINGF,1STEPG,STEPTA,...,Insecta,Hemiptera,Heteroptera,Tingidae,Stephanitis,Stephanitis takeyai,STEPTA,VACSS,Phalaenoptilus nuttallii,0
147417,2339,1ANIMK,1ARTHP,1HEXAQ,1INSEC,1HEMIO,1HETER,1TINGF,1STEPG,STEPTA,...,Insecta,Hemiptera,Heteroptera,Tingidae,Stephanitis,Stephanitis takeyai,STEPTA,VACSS,Poecile gambeli,1
147418,2339,1ANIMK,1ARTHP,1HEXAQ,1INSEC,1HEMIO,1HETER,1TINGF,1STEPG,STEPTA,...,Insecta,Hemiptera,Heteroptera,Tingidae,Stephanitis,Stephanitis takeyai,STEPTA,VACSS,Porzana carolina,1


The table above contains the taxonomic classification of the pests, in EPPO code format (eppocode_?) and taxon name (prefname_?), the EPPO code of the crop (crop_eppocode), the bird species and the boolean column `eats`, which indicates if the bird species eats or not the pest.